In [13]:
import timm
import torch
from  torch import nn

In [2]:
m = timm.create_model('resnet50d', pretrained=False, in_chans = 1)
x =torch.randn(1, 1, 224, 224)
m(x).shape

torch.Size([1, 1000])

In [3]:
m.fc

Linear(in_features=2048, out_features=1000, bias=True)

In [4]:
m.get_classifier()

Linear(in_features=2048, out_features=1000, bias=True)

In [6]:
timm.create_model('resnet50d', pretrained=False, num_classes=10).get_classifier()

Linear(in_features=2048, out_features=10, bias=True)

In [7]:
timm.create_model('resnet50d', pretrained=False, num_classes=0).get_classifier()

Identity()

In [8]:
m.global_pool

SelectAdaptivePool2d (pool_type=avg, flatten=Flatten(start_dim=1, end_dim=-1))

In [9]:
pool_types = ['avg', 'max', 'avgmax', 'catavgmax', '']

for pool in pool_types:
    model = timm.create_model('resnet50d', pretrained=True, num_classes=0, global_pool=pool)
    model.eval()
    feature_output = model(torch.rand(1, 3, 224, 224))
    print(feature_output.shape)

Downloading: "https://github.com/rwightman/pytorch-image-models/releases/download/v0.1-weights/resnet50d_ra2-464e36ba.pth" to C:\Users\zc199/.cache\torch\hub\checkpoints\resnet50d_ra2-464e36ba.pth


torch.Size([1, 2048])
torch.Size([1, 2048])
torch.Size([1, 2048])
torch.Size([1, 4096])
torch.Size([1, 2048, 7, 7])


In [10]:
m = timm.create_model('resnet50d', pretrained=True)
print('Original pooling: {}'.format(m.global_pool))
print('Original classifier: {}'.format(m.get_classifier()))
print('----------------')

m.reset_classifier(10, 'max')
print('Original pooling: {}'.format(m.global_pool))
print('Original classifier: {}'.format(m.get_classifier()))

Original pooling: SelectAdaptivePool2d (pool_type=avg, flatten=Flatten(start_dim=1, end_dim=-1))
Original classifier: Linear(in_features=2048, out_features=1000, bias=True)
----------------
Original pooling: SelectAdaptivePool2d (pool_type=max, flatten=Flatten(start_dim=1, end_dim=-1))
Original classifier: Linear(in_features=2048, out_features=10, bias=True)


In [12]:
model = timm.create_model('resnet50d', pretrained=True, num_classes=10, global_pool='catavgmax')
num_in_features = model.get_classifier().in_features; num_in_features

4096

In [14]:
model.fc = nn.Sequential(
    nn.BatchNorm1d(num_in_features),
    nn.Linear(in_features=num_in_features, out_features=512, bias=True),
    nn.ReLU(),
    nn.BatchNorm1d(512),
    nn.Dropout(0.4),
    nn.Linear(in_features=512, out_features=10, bias=False)   
)